In [1]:
# Import needed modules
import time
import socket
import numpy as np
import random as rn
import matplotlib.pyplot as pt
import pyvisa
import csv

# Header and end required for each eSCL message
header = bytes([0x00, 0x07])
end = bytes([0xD])

# This is the socket used by the motors (shouldnt ever be changed)
UDP_PORT = 7775


DriveIP = "192.168.1.20" # Drive IP can be configured (shouldnt ever be changed)
RouterIP = "192.168.1.4" # Check router to see what IP it assigned your device

rm = pyvisa.ResourceManager() #Finds all open resources
rm.list_resources() #Lists all open resources
VNA = rm.open_resource('USB0::0x0957::0x0D09::MY46214932::INSTR') #connects to the VNA. If this doesnt work, run rm.list_resources() alone and troubleshoot

In [2]:
def command(Message, s):
    encodeMessage = Message.encode()
    toSend = header + encodeMessage + end
    s.sendto(toSend,(DriveIP,UDP_PORT))
   
    #If the motor times out (i.e. does not send a response in 0.1s then Python throws a socket.timeout error)
    #So this code tries to receive a message from the motor, and if the motor sends nothing in 0.1s, it times\
    #out. So python throws a socket.timeout error, and the code handles that by just moving to the next line
    try:
        recMessage = s.recv(1024).decode()
        print(recMessage[2:])
    except socket.timeout:
        pass


    
def movrel(spd,amt): #speed (Hz), number of microsteps
    if spd > 0.2:
        print('Slow Down There Cowboy')
        return
    
    exprt = (amt/20000)/spd # Expected time to complete movement (seconds)
    
    Scmd = 'VE' + str(spd)
    Stcmd = 'FL' + str(amt)
    
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
        sock.bind((RouterIP, UDP_PORT))
        sock.getsockname()    
        sock.settimeout(.1)          # Proceed to the next command if the motor doesn't send any response in 0.1s
        command('MR8',sock)          # Sets microstep resolution to 20000 steps / rev
        command('DI1', sock)         # Sets the direction to CW and position to ?
        command(Scmd, sock)          # Set jog speed (rev/sec)
        command(Stcmd, sock)         # Move by N steps
        time.sleep(abs(exprt)*1.05)  # Adding a 5% delay after movement
        sock.close()

## Inputs
Enter your center and span (full span, so span/2 on each side), number of positions to sample, number of sweep points, range direction and a file direction

(Note the title string in the program must be edited the first time it is run on a computer)

In [3]:
### Inputs
fcenter = 3    #GHz
span = 25      #MHz
pos = 20       # number of positions
swpts = 400    # Sweep points on VNA
rang = 20      #In
direction = 'down'
name = 'TEST2' #Name of the saved file

In [5]:
# BEGINNING OF PROGRAM

if direction.lower() == 'up':
    d = -1
elif direction.lower() == 'down':
    d = 1
else:
    print('Sorry buckaroo, your options are up and down')

titlstr = '/Users/nolan/Documents/' +name + '.csv'  ##REPLACE WITH INTENDED FILE PATH

#SET WINDOW
centstr = 'SENS:FREQ:CENT' + str(fcenter) + 'E9'
spanstr = 'SENS:FREQ:SPAN' + str(span) + 'E6'
swestr = 'SENS:SWE:POIN ' + str(swpts)

VNA.write(centstr)
VNA.write(spanstr)
VNA.write(swestr)
VNA.write('DISP:WIND:TRAC:Y:AUTO')
VNA.write('CALC:PAR:SEL')
VNA.write('CALC:MARK:FUNC:TYPE MAX') #I'm not sure if this line is necesary but taking it out MIGHT break something

radius = 0.8125 #radius of pully in in
dperstep = rang/pos #distance per step
circ = 2*3.14159*radius
numsteps = round(dperstep/(circ/20000))

data = []
for i in range(1,pos+1):
    movrel(0.2,d*numsteps)
    time.sleep(0.5) #Change this sleep command if you want a different amount of time before sampling
    VNA.write('CALC:MARK:FUNC:EXEC') #finds the maximum
    bwstr = VNA.query('CALC:MARK:BWID:DATA?') #See Page 1061 for details
    bw = float(bwstr[21:39])
    Q = float(float(bwstr[40:59]))
    data.append((i,bw,Q))
    
    #Note it is saving the center frequency of the highest peak, as well as the corresponding Q value
    #If you want to look for something else, here is your place to do so
    
    with open(titlstr, 'a', newline='') as csvfile:
        fieldnames = ['Step','Center','Q']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'Step':(i+1),'Center':bw,'Q':Q})
    csvfile.close()
    
    
# END OF PROGRAM